### **README**

# What this does

This script allows you to download a **dashboard** with any number of tiles as a series of either `csv` or an `xlsx` files instead of the default `csv_zip` that the UI allows for. Can be easily modified to support `json` or `txt` extensions (see Notes)

# Notes
`DownloadCSVorExcel(dash_id, file_name, extension)` accepts 3 inputs:

- `dash_id`(string) - the ID of the dashboard to download 
- `file_name`(string) - the desired output name of the files. Each file will be named `file_name + i`
- `extension`(string) - the desired output extension. Accepts either `xlsx` or `csv`. 

If json or txt extensions are desired, just add additional loops that either fetch results from `run_query` in that format or convert via the script. 

If using colab version, you will need to input your api url, client_id, and client_secret in the `## colab inputs ##` section. 

```
os.environ['LOOKERSDK_BASE_URL'] = 'https://example.looker.com'
os.environ['LOOKERSDK_API_VERSION'] = '3.1'
os.environ['LOOKERSDK_CLIENT_ID'] = 'client id here'
os.environ['LOOKERSDK_CLIENT_SECRET'] = 'client secret here'
```

If using python (local) version, these values should be set for the sdk per either an `.ini` file or as an environment variable. See Python SDK documentation for more information. https://pypi.org/project/looker-sdk/

In [1]:
### colab specific ###
# install the sdk
!pip install looker-sdk
# set environ variables
import os

## colab inputs ##
os.environ['LOOKERSDK_BASE_URL'] = ''
os.environ['LOOKERSDK_API_VERSION'] = '3.1'
os.environ['LOOKERSDK_CLIENT_ID'] = ''
os.environ['LOOKERSDK_CLIENT_SECRET'] = ''

     |████████████████████████████████| 327kB 9.8MB/s 


In [2]:
# will need these for downloading the file from colab
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from google.colab import files
from oauth2client.client import GoogleCredentials
# auth stuff for download
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [9]:
### import packages ###
# import and init
import looker_sdk
sdk = looker_sdk.init31("looker.ini")

import pandas as pd
import csv

In [22]:
## inputs ##

dash_id = ''
file_name = ''
extension = 'csv | xlsx'

In [25]:
def DownloadCSVorExcel(dash_id, file_name, extension):

  # variables 
  data = sdk.search_dashboard_elements(dashboard_id=dash_id)
  listlen = len(data)
  query_ids = []

  # helper methods
  def id_getter(data, listlen, query_ids):
    for i in range(listlen):
      query_ids.append(data[i].result_maker.query_id)
    return query_ids
    
  def downloader(extension, data, listlen, query_ids, file_name):
      idlist = id_getter(data, listlen, query_ids)

      if extension == 'xlsx':

        for i in range(listlen):
            resultset = sdk.run_query(query_id=idlist[i],result_format='csv')

            with open('temp.csv', 'w', newline='') as file:
              file.write(resultset)
            
            with pd.ExcelWriter(file_name+' '+str(i)+'.xlsx') as writer:
              df = pd.read_table('temp.csv', delimiter =",",engine='python')
              df.to_excel(writer, sheet_name=('Sheet 1'))
            
            # colab only
            from google.colab import files 
            files.download(file_name+' '+str(i)+'.xlsx')
            # end colab 
            
      elif extension == 'csv':

        for i in range(listlen):
            resultset = sdk.run_query(query_id=idlist[i],result_format='csv')

            with open(file_name+' '+str(i)+ '.csv', 'w', newline='') as file:
              file.write(resultset)

            # colab only
            from google.colab import files 
            files.download(file_name+' '+str(i)+ '.csv')
            # end colab

      else: 
        print('ERROR')
    
  downloader(extension, data, listlen, query_ids, file_name)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>